# Maksymilian Wojnar

# Generowanie obrazów

W implementacji skorzystałem z biblioteki numba, aby przyspieszyć obliczanie obrazów.

In [ ]:
import matplotlib.pyplot as plt
import numpy as np
from numba import jit, njit

n = 512

Generowanie losowych obrazów z udziałem "delta" wszystkich punktów jako czarnych:

In [ ]:
@njit
def random_img(delta):
    img = np.zeros((n, n), dtype=np.int32)
    points = set()

    while len(points) < delta * n * n:
        points.add((np.random.randint(0, n), np.random.randint(0, n)))

    for p in points:
        img[p[0], p[1]] = 1

    return img

Zapis przykładowego sąsiedztwa (G) oraz funkcje kosztu używane w programie:

In [ ]:
dx = np.array([-1, 0, 1, 1, 1, 0, -1, -1])
dy = np.array([1, 1, 1, 0, -1, -1, -1, 0])


@njit
def possible(x, y):
    return 0 <= x < n and 0 <= y < n


@njit
def cost(img, x, y, val):
    blacks = 0
    
    for i in range(len(dx)):
        if possible(x + dx[i], y + dy[i]) and img[x + dx[i], y + dy[i]] == 1:
            blacks += 1
            
    return -blacks if val == 1 else blacks


@njit
def blacks(img, x, y, val):
    return cost(img, x, y, val)


@njit
def whites(img, x, y, val):
    return -cost(img, x, y, val)

Funkcje wybierające punkty do zamiany. W moim programie, przejście do następnego stanu to zamiana dwóch losowych punktów na obrazie, które różnią się kolorem.

In [ ]:
@njit
def random_swap():
    return np.random.randint(0, n - 1), np.random.randint(0, n - 1)


@njit
def get_points(img):
    while True:
        sw = random_swap()
        sw2 = random_swap()
        
        while img[sw[0], sw[1]] == img[sw2[0], sw2[1]]:
            sw2 = random_swap()
            
        return sw, sw2

Obliczanie temperatury początkowej analogiczne do zadania 3 (praca Rhyd'a Lewis'a "Metaheuristics can Solve Sudoku Puzzles"), jako odchylenie standardowe funkcji kosztu po kilkudziesięciu przejściach po stanach sąsiednich:

In [ ]:
@njit
def calculate_temp(img, cost_func):
    values = np.zeros(20)

    for i in range(1, 20):
        swap1, swap2 = get_points(img)
        val = img[swap1[0], swap1[1]]
        
        new_energy = cost_func(img, swap1[0], swap1[1], img[swap2[0], swap2[1]])
        new_energy += cost_func(img, swap2[0], swap2[1], val)
        
        values[i] = values[i - 1] + new_energy

    return np.std(values)

Główna funkcja homogenicznego symulowanego wyżarzania z pamięcią najlepszego wyniku oraz różnymi funkcjami spadku temperatury:

In [ ]:
@njit
def anneal(img, cost_func, temp_type='f'):
    t = temp = calculate_temp(img, cost_func)
    
    best_val = last = 0
    best_img = img.copy()

    for i in range(500):
        for _ in range(150000):
            swap1, swap2 = get_points(img)
            val = img[swap1[0], swap1[1]]
            
            new_energy = cost_func(img, swap1[0], swap1[1], img[swap2[0], swap2[1]])
            new_energy += cost_func(img, swap2[0], swap2[1], val)
            
            img[swap1[0], swap1[1]], img[swap2[0], swap2[1]] = img[swap2[0], swap2[1]], img[swap1[0], swap1[1]]

            if np.exp(-new_energy / t) > np.random.random():
                last += new_energy
            else:
                img[swap1[0], swap1[1]], img[swap2[0], swap2[1]] = img[swap2[0], swap2[1]], img[swap1[0], swap1[1]]

        if last < best_val:
            best_val = last
            best_img = img.copy()
                
        if temp_type == 'f':
            t *= 0.98
        elif temp_type == 's':
            t *= 0.999
        elif temp_type == 'l':
            t = -i * temp / 500 + temp

    return best_img

Generowanie obrazów:

In [ ]:
for delta, d_name in zip([0.1, 0.3, 0.4], ['1', '3', '4']):
    for cost_func, f_name in zip([whites, blacks], ['W', 'B']):
        img = anneal(random_img(delta), cost_func)

        plt.figure(figsize=(8, 8), dpi=200)
        plt.imshow(img, cmap='Greys', interpolation='nearest')
        plt.savefig(f"./images/T/{f_name}{d_name}")
        plt.show()

## Wybór funkcji kosztu oraz sąsiedztw

 - Wybór funkcji kosztu oraz sąsiedztwa miał kluczowy wypływ na efekt uzyskany na obrazie, dlatego w swojej pracy zaproponowałem wiele różnych sąsiedztw i kilka funkcji. Obrazy wygenerowane za pomocą poszczególnych kombinacji znajdują się w folderze "images" w podfolderach od "A" do "S".
 
 - W folderach od "A" do "O" znajdują się obrazy generowane za pomocą różnych sąsiedztw. Każde sąsiedztwo jest opisane przez obrazek "neighbourhood.png" znajdujący się w danym katalogu. Na niebiesko jest zaznaczony aktualnie rozpatrywany punkt, a na szaro wybrane sąsiedztwo. Obrazy te generowałem z użyciem dwóch funkcji kosztu, które zamieściłem w kodzie powyżej - "blacks" oraz "whites" (oznaczyłem je w ten sposób, gdyż "blacks" powoduje, że czarne punkty przyciągają się, natomiast "whites" sprawia, iż czarne punkty odpychają się). Obrazy wygenerowane z funkcją "blacks" mają nazwę rozpoczynającą się na "B", w przeciwnym wypadku rozpoczynają się na "W". Cyfra 1, 3 lub 4 na końcu nazwy pliku to oznaczenie gęstości czarnych punktów na obrazie (delta = {0.1, 0.3, 0.4}).
 
 - W folderach od "P" do "S" są obrazy, w których funkcja energii każdego punktu nie zależała od sąsiedztwa, lecz od jego współrzędnych na obrazie. W tych katalogach zamieściłem plik "energy.txt", w którym opisałem krótko używaną funkcję. Oznaczenia obrazów są analogiczne do tych opisanych wyżej.
 
 - Kopie najlepszych (w mojej opinii) obrazów zamieściłem w katalogu "Wybrane", aby umożliwić szybkie zobaczenie wyników mojej pracy, bez konieczności przeglądania wszystkich folderów od "A" do "S". 

## Wpływ szybkości spadku temperatury 

 - W ramach testów wpływu spadku temperatury na obrazy, wygenerowałem w katalogu "Spadek temperatury" przykładowe obrazy w oparciu o sąsiedztwo M. Mają oznaczenia podobne do obrazów w innych katalogach, z dodatkową literą na końcu ("F" - szybki spadek wykładniczy ( t *= 0.98 ), "S" - wolny spadek wykłaniczy ( t *= 0.999 ), "L" - spadek liniowy ( t = -i * temp / 500 + temp ), "N" - brak spadku).
 
 - Szybki spadek wykładniczy oraz spadek liniowy dawały w ogólności podobne rezultaty. Można jednak zauważyć, że w przypadku generowania z funkcją "blacks", spadek liniowy dawał większe obszary połączonych czarnych punktów, natomiast spadek wykładniczy podowował większe rozdrobnienie. Może to mieć związek z szybszym spadkiem wykładniczym na początku, ale wolniejszym na końcu procesu, w stosunku do spadku liniowego.
 
 - Wolny spadek wykładniczy i brak spadku temperatury powodowały znaczne zniekształcenie obrazów. Widać to szczególnie na obrazach "B4N.png", "B4S.png", "B3N.png" i "B3S.png". Duża temperatura w trakcie całego procesu wyżarzania sprawiła, że na obrazach nie "wykrystalizowały" się białe oraz czarne plamy, natomiast jest na nich sporo szumu. Podobnie na "W3N.png", "W3S.png", "W4N.png" oraz "W4S.png" - w tym wypadku widać jedynie zarys formowania się pewnej struktury, lecz daleko jeszcze do osiągnięcia ostatecznego stanu, jak przy liniowym lub szybkim spadku wykładniczym.